# 📊 Modélisation Formelle du Problème de Tournées de Véhicules

## Projet ADEME - Optimisation des Tournées de Livraison

**Équipe CesiCDP** | **Date :** Octobre 2025

---

## 📋 Table des Matières

1. [Introduction et Contexte](#1-introduction-et-contexte)
2. [Formulation Mathématique de Base](#2-formulation-mathématique-de-base)
3. [Contraintes Supplémentaires](#3-contraintes-supplémentaires)
4. [Analyse de Complexité](#4-analyse-de-complexité)
5. [Représentation des Données](#5-représentation-des-données)
6. [Références Bibliographiques](#6-références-bibliographiques)

## 1. Introduction et Contexte

### 1.1 Contexte ADEME

Dans le cadre des engagements environnementaux de la France (réduction de 4x des émissions d'ici 2050), l'ADEME (Agence de l'Environnement et de la Maîtrise de l'Énergie) a lancé un appel à manifestation d'intérêt pour développer des solutions de mobilité intelligente.

### 1.2 Problématique

Le **Vehicle Routing Problem (VRP)** consiste à déterminer un ensemble optimal de routes pour une flotte de véhicules afin de servir un ensemble de clients, en minimisant les coûts (distance, temps, émissions) tout en respectant diverses contraintes opérationnelles.

### 1.3 Applications Industrielles

- 📦 **Livraison e-commerce** : Amazon, FedEx, DHL
- 🏪 **Distribution retail** : Carrefour, Leclerc
- 🚛 **Transport de marchandises** : optimisation des tournées inter-entrepôts
- ♻️ **Collecte des déchets** : optimisation des circuits de ramassage

**Impact potentiel :** Réduction de 15-25% des distances parcourues = baisse significative des émissions CO₂

## 2. Formulation Mathématique de Base

### 2.1 Problème de Base : CVRP (Capacitated Vehicle Routing Problem)

#### Données d'entrée

- $G = (V, A)$ : graphe complet où $V = \{0, 1, 2, ..., n\}$ avec $0$ = dépôt
- $c_{ij}$ : coût (distance/temps) entre les nœuds $i$ et $j$
- $d_i$ : demande du client $i$ ($d_0 = 0$)
- $K$ : nombre de véhicules disponibles
- $Q$ : capacité uniforme des véhicules

#### Variables de décision

$$x_{ij}^k = \begin{cases}
1 & \text{si le véhicule } k \text{ emprunte l'arc } (i,j) \\
0 & \text{sinon}
\end{cases}$$

#### Fonction objectif

$$\min \sum_{k=1}^{K} \sum_{i=0}^{n} \sum_{j=0}^{n} c_{ij} \cdot x_{ij}^k$$

#### Contraintes principales

**Visite unique de chaque client :**
$$\sum_{k=1}^{K} \sum_{j=0}^{n} x_{ij}^k = 1, \quad \forall i \in \{1, ..., n\}$$

**Conservation des flots :**
$$\sum_{i=0}^{n} x_{ij}^k = \sum_{i=0}^{n} x_{ji}^k, \quad \forall j \in V, \forall k$$

**Départ/retour au dépôt :**
$$\sum_{j=1}^{n} x_{0j}^k = \sum_{i=1}^{n} x_{i0}^k \leq 1, \quad \forall k$$

**Contrainte de capacité :**
$$\sum_{i=1}^{n} d_i \sum_{j=0}^{n} x_{ij}^k \leq Q, \quad \forall k$$

**Élimination des sous-tours :**
$$\sum_{i \in S} \sum_{j \in S} x_{ij}^k \leq |S| - 1, \quad \forall S \subset \{1, ..., n\}, |S| \geq 2, \forall k$$

## 3. Contraintes Supplémentaires

### 3.1 Fenêtres Temporelles (VRPTW)

#### Données supplémentaires
- $[e_i, l_i]$ : fenêtre temporelle du client $i$
- $s_i$ : temps de service au client $i$
- $t_{ij}$ : temps de trajet entre $i$ et $j$

#### Variables supplémentaires
- $T_i^k$ : temps d'arrivée du véhicule $k$ au client $i$

#### Contraintes temporelles

**Respect des fenêtres temporelles :**
$$e_i \leq T_i^k \leq l_i, \quad \forall i \in V, \forall k$$

**Cohérence temporelle :**
$$T_j^k \geq T_i^k + s_i + t_{ij} - M(1 - x_{ij}^k), \quad \forall (i,j) \in A, \forall k$$

où $M$ est une grande constante.

#### Version avancée : Attente autorisée

**Variable d'attente :**
$$W_i^k = \max(0, e_i - T_i^k)$$

**Fonction objectif modifiée :**
$$\min \sum_{k,i,j} c_{ij} x_{ij}^k + \alpha \sum_{k,i} W_i^k$$

où $\alpha$ est le coût d'attente.

### 3.2 Flotte Hétérogène (HFVRP)

#### Données supplémentaires
- $K_m$ : nombre de véhicules de type $m$
- $Q_m$ : capacité des véhicules de type $m$
- $f_m$ : coût fixe d'utilisation du type $m$

#### Variables supplémentaires
$$y_k^m = \begin{cases}
1 & \text{si le véhicule } k \text{ est de type } m \\
0 & \text{sinon}
\end{cases}$$

#### Contraintes de flotte

**Affectation unique :**
$$\sum_{m} y_k^m \leq 1, \quad \forall k$$

**Disponibilité par type :**
$$\sum_{k} y_k^m \leq K_m, \quad \forall m$$

**Capacité variable :**
$$\sum_{i=1}^{n} d_i \sum_{j=0}^{n} x_{ij}^k \leq \sum_{m} Q_m y_k^m, \quad \forall k$$

#### Version avancée : Compatibilité produit-véhicule

**Matrice de compatibilité :**
$$\text{comp}_{i,m} = \begin{cases}
1 & \text{si le client } i \text{ peut être servi par un véhicule de type } m \\
0 & \text{sinon}
\end{cases}$$

**Contrainte de compatibilité :**
$$\sum_{j} x_{ij}^k \leq \sum_{m} \text{comp}_{i,m} \cdot y_k^m, \quad \forall i, k$$

### 3.3 Trafic Dynamique (TD-VRP)

#### Principe
Les temps de trajet varient selon l'heure de la journée pour refléter les conditions de trafic.

#### Données supplémentaires
- $H$ : ensemble des créneaux horaires
- $t_{ij}^h$ : temps de trajet entre $i$ et $j$ pendant le créneau $h$

#### Variables supplémentaires
$$z_{ij}^{kh} = \begin{cases}
1 & \text{si le véhicule } k \text{ emprunte } (i,j) \text{ pendant le créneau } h \\
0 & \text{sinon}
\end{cases}$$

#### Contraintes temporelles dynamiques

**Cohérence arc-créneau :**
$$\sum_{h} z_{ij}^{kh} = x_{ij}^k, \quad \forall (i,j), k$$

**Temps de trajet variable :**
$$T_j^k \geq T_i^k + s_i + \sum_{h} t_{ij}^h z_{ij}^{kh} - M(1 - x_{ij}^k)$$

#### Fonction objectif temporelle
$$\min \sum_{k,i,j,h} t_{ij}^h \cdot z_{ij}^{kh} + \text{pénalités retard}$$

### 3.4 Points de Collecte Multiples (MDVRP)

#### Extension multi-dépôts
- $D = \{0_1, 0_2, ..., 0_p\}$ : ensemble des dépôts
- Chaque client peut avoir un dépôt de collecte spécifique

#### Contraintes additionnelles

**Affectation dépôt-véhicule :**
$$\sum_{d \in D} \sum_{j=1}^{n} x_{dj}^k \leq 1, \quad \forall k$$

**Retour au même dépôt :**
$$\sum_{j=1}^{n} x_{dj}^k = \sum_{i=1}^{n} x_{id}^k, \quad \forall d \in D, k$$

## 4. Analyse de Complexité

### 4.1 Complexité Théorique

**Théorème :** Le problème VRP est NP-difficile.

**Preuve :** Réduction depuis le problème du voyageur de commerce (TSP).

Soit une instance TSP avec $n$ villes. On construit une instance VRP équivalente :
- Même graphe $G = (V, A)$
- $K = 1$ véhicule
- $Q = n$ (capacité suffisante)
- $d_i = 1$ pour tout $i \neq 0$

La solution optimale du VRP correspond exactement à la solution optimale du TSP.

### 4.2 Complexité selon les Variantes

| Variante | Complexité | Espace de solutions |
|----------|------------|--------------------|
| TSP | $O(n!)$ | $(n-1)!/2$ tours |
| CVRP | $O(n! \cdot K^n)$ | Exponentielle en $n$ et $K$ |
| VRPTW | NP-difficile | Contraintes temporelles réduisent l'espace |
| HFVRP | NP-difficile | Explosion combinatoire avec types de véhicules |

### 4.3 Limites Pratiques

**Résolution exacte :**
- Méthodes exactes : $n \leq 50-100$ clients
- Branch-and-Cut : jusqu'à 200 clients

**Métaheuristiques :**
- Recuit simulé : $n \leq 1000$ clients
- Recherche tabou : $n \leq 2000$ clients
- ALNS : $n \leq 5000$ clients

**Justification du choix métaheuristique :**
Pour des instances industrielles avec $n > 1000$, les métaheuristiques sont le seul moyen d'obtenir des solutions de qualité en temps raisonnable.

## 5. Représentation des Données

### 5.1 Structure d'Instance

In [ ]:
import numpy as np
from typing import Dict, List, Tuple, Optional
import matplotlib.pyplot as plt

class VRPInstance:
    """
    Représentation d'une instance VRP avec contraintes multiples.
    """
    
    def __init__(self, name: str):
        self.name = name
        self.dimension = 0  # Nombre total de nœuds (incluant dépôt)
        self.depot = 0      # ID du dépôt
        
        # Données géographiques
        self.node_coords: Dict[int, Tuple[float, float]] = {}
        self.distance_matrix: Optional[np.ndarray] = None
        
        # Demandes clients
        self.demands: Dict[int, int] = {}
        
        # Contraintes de capacité
        self.vehicle_count = 1
        self.capacity = 100
        self.vehicle_capacities: List[int] = []
        
        # Fenêtres temporelles [début, fin]
        self.time_windows: Dict[int, Tuple[int, int]] = {}
        self.service_times: Dict[int, int] = {}
        
        # Trafic dynamique
        self.time_slots = 1
        self.dynamic_distances: List[np.ndarray] = []
        
    def add_customer(self, customer_id: int, x: float, y: float, demand: int,
                     time_window: Optional[Tuple[int, int]] = None,
                     service_time: int = 0):
        """Ajouter un client à l'instance."""
        self.node_coords[customer_id] = (x, y)
        self.demands[customer_id] = demand
        
        if time_window:
            self.time_windows[customer_id] = time_window
        if service_time > 0:
            self.service_times[customer_id] = service_time
            
        self.dimension = max(self.dimension, customer_id + 1)
    
    def calculate_distance_matrix(self):
        """Calculer la matrice des distances euclidiennes."""
        nodes = sorted(self.node_coords.keys())
        n = len(nodes)
        self.distance_matrix = np.zeros((n, n))
        
        for i, node_i in enumerate(nodes):
            for j, node_j in enumerate(nodes):
                if i != j:
                    x1, y1 = self.node_coords[node_i]
                    x2, y2 = self.node_coords[node_j]
                    distance = np.sqrt((x1 - x2)**2 + (y1 - y2)**2)
                    self.distance_matrix[i][j] = distance
    
    def visualize(self, figsize=(10, 8)):
        """Visualiser l'instance."""
        plt.figure(figsize=figsize)
        
        # Tracer les clients
        customers = [(x, y) for node, (x, y) in self.node_coords.items() if node != self.depot]
        if customers:
            cx, cy = zip(*customers)
            plt.scatter(cx, cy, c='blue', s=60, alpha=0.7, label='Clients')
        
        # Tracer le dépôt
        if self.depot in self.node_coords:
            dx, dy = self.node_coords[self.depot]
            plt.scatter(dx, dy, c='red', s=120, marker='s', label='Dépôt')
        
        # Annotations
        for node, (x, y) in self.node_coords.items():
            if node != self.depot:
                demand = self.demands.get(node, 0)
                plt.annotate(f'{node}({demand})', (x, y), xytext=(5, 5), 
                           textcoords='offset points', fontsize=8)
        
        plt.title(f'Instance VRP: {self.name}')
        plt.xlabel('Coordonnée X')
        plt.ylabel('Coordonnée Y')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.show()

# Exemple d'utilisation
print("✅ Structure VRPInstance définie")

### 5.2 Exemple d'Instance

In [ ]:
# Création d'une instance exemple
instance = VRPInstance("Exemple-5-clients")

# Définir le dépôt
instance.node_coords[0] = (50, 50)  # Dépôt au centre
instance.demands[0] = 0
instance.depot = 0

# Ajouter les clients avec fenêtres temporelles
clients_data = [
    (1, 20, 30, 10, (8, 12)),   # Client 1: demande=10, fenêtre 8h-12h
    (2, 80, 70, 15, (9, 14)),   # Client 2: demande=15, fenêtre 9h-14h
    (3, 30, 80, 8, (10, 16)),   # Client 3: demande=8, fenêtre 10h-16h
    (4, 70, 20, 12, (11, 15)),  # Client 4: demande=12, fenêtre 11h-15h
    (5, 40, 60, 9, (13, 17)),   # Client 5: demande=9, fenêtre 13h-17h
]

for customer_id, x, y, demand, time_window in clients_data:
    instance.add_customer(customer_id, x, y, demand, time_window, service_time=1)

# Paramètres de flotte
instance.vehicle_count = 2
instance.capacity = 25

# Calculer les distances
instance.calculate_distance_matrix()

# Afficher les informations
print(f"Instance: {instance.name}")
print(f"Nombre de clients: {len(instance.demands) - 1}")
print(f"Capacité véhicule: {instance.capacity}")
print(f"Demande totale: {sum(d for i, d in instance.demands.items() if i != 0)}")

# Visualiser
instance.visualize()

### 5.3 Matrice de Distances

In [ ]:
# Afficher la matrice de distances
import pandas as pd

nodes = sorted(instance.node_coords.keys())
distance_df = pd.DataFrame(
    instance.distance_matrix, 
    index=[f"Nœud {i}" for i in nodes],
    columns=[f"Nœud {i}" for i in nodes]
)

print("📏 Matrice des distances:")
print(distance_df.round(2))

# Statistiques
distances = instance.distance_matrix[instance.distance_matrix > 0]
print(f"\n📊 Statistiques des distances:")
print(f"Distance moyenne: {distances.mean():.2f}")
print(f"Distance min: {distances.min():.2f}")
print(f"Distance max: {distances.max():.2f}")

### 5.4 Analyse des Contraintes

In [ ]:
# Analyse des fenêtres temporelles
print("🕐 Analyse des fenêtres temporelles:")
print("-" * 50)

for customer, (start, end) in instance.time_windows.items():
    duration = end - start
    demand = instance.demands[customer]
    print(f"Client {customer}: [{start:2d}h - {end:2d}h] (durée: {duration}h, demande: {demand})")

# Contraintes de capacité
total_demand = sum(d for i, d in instance.demands.items() if i != 0)
min_vehicles = np.ceil(total_demand / instance.capacity)

print(f"\n🚛 Analyse de capacité:")
print("-" * 30)
print(f"Demande totale: {total_demand}")
print(f"Capacité par véhicule: {instance.capacity}")
print(f"Véhicules disponibles: {instance.vehicle_count}")
print(f"Véhicules minimum requis: {int(min_vehicles)}")

if instance.vehicle_count >= min_vehicles:
    print("✅ Contraintes de capacité: RÉALISABLES")
else:
    print("❌ Contraintes de capacité: NON RÉALISABLES")

## 6. Références Bibliographiques

### 6.1 Ouvrages de Référence

1. **Toth, P., & Vigo, D. (2014).** *Vehicle Routing: Problems, Methods, and Applications* (2nd ed.). SIAM.
   - Référence principale sur les problèmes de tournées
   - Couvre l'ensemble des variantes et méthodes de résolution

2. **Golden, B. L., Raghavan, S., & Wasil, E. A. (2008).** *The vehicle routing problem: latest advances and new challenges*. Springer.
   - État de l'art sur les avancées récentes
   - Applications industrielles

### 6.2 Articles Fondamentaux

3. **Clarke, G., & Wright, J. W. (1964).** "Scheduling of vehicles from a central depot to a number of delivery points." *Operations Research*, 12(4), 568-581.
   - Algorithme des économies (savings algorithm)
   - Base historique du VRP

4. **Solomon, M. M. (1987).** "Algorithms for the vehicle routing and scheduling problems with time window constraints." *Operations Research*, 35(2), 254-265.
   - Instances de référence pour VRPTW
   - Heuristiques pour fenêtres temporelles

### 6.3 Métaheuristiques

5. **Cordeau, J. F., Gendreau, M., & Laporte, G. (1997).** "A tabu search heuristic for periodic and multi-depot vehicle routing problems." *Networks*, 30(2), 105-119.
   - Recherche tabou pour VRP
   - Extensions multi-dépôts

6. **Pisinger, D., & Ropke, S. (2007).** "A general heuristic for vehicle routing problems." *Computers & Operations Research*, 34(8), 2403-2435.
   - Adaptive Large Neighborhood Search (ALNS)
   - Méthode générique pour différentes variantes

### 6.4 Applications Environnementales

7. **Bektaş, T., & Laporte, G. (2011).** "The pollution-routing problem." *Transportation Research Part B*, 45(8), 1232-1250.
   - Optimisation avec critères environnementaux
   - Minimisation des émissions CO₂

### 6.5 Benchmarks et Instances

8. **Augerat, P., et al. (1995).** "Computational results with a branch and cut code for the capacitated vehicle routing problem." *Technical Report*, INRIA.
   - Instances de référence (Set A, B, E)
   - Solutions optimales connues

9. **VRP-REP (2020).** "Vehicle Routing Problem Repository." Accessible: http://vrp.galgos.inf.puc-rio.br/
   - Base de données d'instances standardisées
   - Résultats de référence pour comparaisons

### 6.6 Outils et Bibliothèques

10. **VRPLib (2023).** "Python library for Vehicle Routing Problem instances." Accessible: https://vrplib.readthedocs.io/
    - Bibliothèque Python pour instances VRP
    - Interface standardisée pour benchmarks

---

## 📋 Résumé de la Modélisation

### Modèle de Base Sélectionné
**CVRPTW (Capacitated Vehicle Routing Problem with Time Windows)** avec extensions :

1. ✅ **Fenêtres temporelles** avec attente autorisée
2. ✅ **Flotte hétérogène** avec contraintes de compatibilité
3. 🔄 **Trafic dynamique** (en développement)
4. 🔄 **Multi-dépôts** (extension future)

### Justification du Choix
- **Pertinence industrielle** : Contraintes réalistes pour livraisons urbaines
- **Impact environnemental** : Optimisation temps/distance réduit les émissions
- **Complexité maîtrisée** : Métaheuristiques adaptées aux grandes instances
- **Validation possible** : Benchmarks disponibles via VRPLib

### Prochaines Étapes
1. **Implémentation** des algorithmes métaheuristiques
2. **Validation** sur instances de référence
3. **Analyse expérimentale** comparative
4. **Étude d'impact** environnemental

---

*Ce notebook constitue le livrable de modélisation formelle pour le projet ADEME. Il sera complété par les notebooks d'implémentation et d'analyse expérimentale.*